<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Установка библиотек

In [34]:
# pip install nltk # доустанавливаем необходимые библиотеки
# pip install pywsd
# pip install transformers

### Импорт библиотек

In [35]:
import pandas as pd
import torch
import transformers 
import transformers as ppb # pytorch transformers

import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

from tqdm import notebook

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

from sklearn.metrics import f1_score

from urllib.parse import urlencode # для задания адреса таблицы в я.диске
import requests #получить финальный url я.диска

import nltk
import re 
from pywsd.utils import lemmatize_sentence #лемматизатор
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer 


### Подготовка констант

In [36]:
RANDOM_STATE = 100
max_len = 512

### Знакомство с данными

In [37]:
try:
    df = pd.read_csv('toxic_comments.csv')
except:
    df = pd.read_csv(r'C:\Users\maxpe\Downloads\Practicum\Projects\datasets\toxic_comments.csv')
display(df.shape);

(159292, 3)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [39]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Датасет не содержит пустых строк, состоит из 3 столбцов, один из которых будет преобразован в признаки (`text`), а `toxic` - целевой признак. Столбец `Unnamed` можем удалить. 

In [40]:
df = df.drop(['Unnamed: 0'], axis=1)

## BERT

### Подготовка данных для BERT

#### Загрузка предобученных модели и токенизатора.

In [41]:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights);

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Проверим любой рандомный текст.

In [42]:
tokenizer.encode('You, sir, are my hero. Any chance you remember..', add_special_tokens=True)

[101,
 2017,
 1010,
 2909,
 1010,
 2024,
 2026,
 5394,
 1012,
 2151,
 3382,
 2017,
 3342,
 1012,
 1012,
 102]

Токенизатор отрабатывает корректно. Применим его предварительно к датасету.

#### Преобразуем текст

In [43]:
pre_tokenized = df['text'].apply(
  lambda x: tokenizer.encode(x, add_special_tokens=True)) 

Token indices sequence length is longer than the specified maximum sequence length for this model (631 > 512). Running this sequence through the model will result in indexing errors


In [44]:
pre_tokenized

0         [101, 7526, 2339, 1996, 10086, 2015, 2081, 210...
1         [101, 1040, 1005, 22091, 2860, 999, 2002, 3503...
2         [101, 4931, 2158, 1010, 1045, 1005, 1049, 2428...
3         [101, 1000, 2062, 1045, 2064, 1005, 1056, 2191...
4         [101, 2017, 1010, 2909, 1010, 2024, 2026, 5394...
                                ...                        
159287    [101, 1000, 1024, 1024, 1024, 1024, 1024, 1998...
159288    [101, 2017, 2323, 2022, 14984, 1997, 4426, 200...
159289    [101, 13183, 6290, 26114, 1010, 2045, 2015, 20...
159290    [101, 1998, 2009, 3504, 2066, 2009, 2001, 2941...
159291    [101, 1000, 1998, 1012, 1012, 1012, 1045, 2428...
Name: text, Length: 159292, dtype: object

Как видим из сообщения к `pre_tokenized`, имеем ограничение в 512 токенов в строке. Создадим такое ограничение.

#### Фильтруем данные

Собираем датасет из токенизированных данных - столбец `text`, целевого признака и длин списков токенов.

In [45]:
df_temp = pd.DataFrame(pre_tokenized)
df_temp['toxic'] = df['toxic']
len_list = []
for i in df_temp['text']:
    len_list.append(len(i))
df_temp['len'] = len_list
df_temp

,text,toxic,len
0,"[101, 7526, 2339, 1996, 10086, 2015, 2081, 210...",0,68
1,"[101, 1040, 1005, 22091, 2860, 999, 2002, 3503...",0,35
2,"[101, 4931, 2158, 1010, 1045, 1005, 1049, 2428...",0,54
3,"[101, 1000, 2062, 1045, 2064, 1005, 1056, 2191...",0,144
4,"[101, 2017, 1010, 2909, 1010, 2024, 2026, 5394...",0,21
...,...,...,...
159287,"[101, 1000, 1024, 1024, 1024, 1024, 1024, 1998...",0,68
159288,"[101, 2017, 2323, 2022, 14984, 1997, 4426, 200...",0,27
159289,"[101, 13183, 6290, 26114, 1010, 2045, 2015, 20...",0,19
159290,"[101, 1998, 2009, 3504, 2066, 2009, 2001, 2941...",0,28


Удаляем строки с превышающими лимит токенами. таких строк порядка 1.8%

In [46]:
df_temp = df_temp.loc[df_temp['len'] < 513].drop(['len'], axis=1)

Собираем данные по индексу: слева - тексты из исходного датасета, справа - целевой признак из нового, отфильтрованного датасета. Так как `join` имеет значение `inner`, то остаются только те строки, в которых совпадает индекс, то есть прошедшие фильтрацию.

In [47]:
df_left = df['text']
df_right = df_temp['toxic']
result = pd.concat([df_left, df_right], axis=1, join="inner")
result

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159287,""":::::And for the second time of asking, when ...",0
159288,You should be ashamed of yourself \n\nThat is ...,0
159289,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,And it looks like it was actually you who put ...,0


В случае преобразования в эмбенддинги внутри ноутбука, код ниже - указывает количество семплов для будущего датасета, где `q_sample = len(result)` - это полный датасет.

Ниже приведен пример расчета для очень маленькой выборки - взяли 100 строк, чтобы показать работоспособность кода. 

Полный преобразованный датасет будет использоваться уже на этапе разделения признаков и целевого признака.

In [48]:
q_sample = 100 # нужное число для датасета, вплоть до q_sample = len(result)
result = result.sample(q_sample).reset_index(drop=True)

#### Преобразуем текст предобработанных данных

Токенизируем отфильтрованную выборку.

In [49]:
tokenized = result['text'].apply(
  lambda x: tokenizer.encode(x, add_special_tokens=True)) 

Предупреждения больше нет. Максимальная длина токена `max_len` равна ограничению - 512. Вынесли это значения в константы.

При исполнении преобразования внутри ноутбука, `max_len` расчитывается ниже. Так как мы случайным образом отберем какое-то количество строк, может случиться, что строки будут короче константной `max_len = 512`, это нужно учитывать

In [50]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
max_len

427

#### Преобразование векторов

Паддингом добавляем к каждому вектору нули в конец, так, чтобы длина каждого вектора была равна длине `max_len`.

Маска тоже применяется к каждому вектору - значению, отличному от нуля, присваивается 1, нули остаются нулями.

In [51]:
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)

Заранее рассчитанные данные:
- В Яндекс.Диске лежит датасет, содержащий 78% данных (ограничение по размеру файла), при батче равном 200. Он будет прочитан позже.

Преобразование эмбеддингов было произведено в Google Colab, с использованием GPU.

#### Преобразование в эмбеддинги

In [52]:
batch_size = 50 # задаем размер батча (кратен q_sample, если уменьшаем q_sample - уменьшаем и батч)
embeddings = [] # список эмбеддингов
for i in notebook.tqdm(range(padded.shape[0] // batch_size)): # цикл по батчам, отображать прогресс будет функция notebook()
        batch = torch.IntTensor(padded[batch_size*i:batch_size*(i+1)]) # преобразуем данные в формат тензоров
        attention_mask_batch = torch.IntTensor(attention_mask[batch_size*i:batch_size*(i+1)]) # преобразуем маску
        
        with torch.no_grad(): # для ускорения вычисления функцией no_grad() в библиотеке torch укажем, что градиенты не нужны
            batch_embeddings = model(batch, attention_mask=attention_mask_batch) # чтобы получить эмбеддинги для батча, передадим модели данные и маску
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy()) # преобразуем элементы методом numpy() к типу numpy.array

df_ed = pd.DataFrame(np.concatenate(embeddings)) # соберём все эмбеддинги в матрицу признаков вызовом функции concatenate()
df_ed['toxic'] = result['toxic'] # добавляем целевой признак
df_ed

  0%|          | 0/2 [00:00<?, ?it/s]

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,toxic
0,0.180258,-0.069345,0.067308,0.129491,0.527740,-0.173354,-0.136002,0.456516,-0.261843,0.051326,...,0.232042,0.065451,-0.136329,-0.036738,0.351970,-0.020417,-0.090589,0.355534,0.694603,0
1,0.257796,0.173529,-0.096371,0.362834,0.370819,-0.450347,0.213386,0.352308,-0.219311,-0.073698,...,0.227018,0.164091,-0.001414,-0.043954,0.117454,-0.126218,0.072296,0.454344,0.253548,0
2,-0.265842,0.079536,-0.008228,0.050172,0.048826,-0.294324,0.248901,0.595242,-0.146441,-0.640853,...,-0.497844,0.252351,-0.408397,0.076342,0.369730,-0.198952,0.018505,0.258208,0.606681,0
3,0.392533,-0.003290,-0.242121,-0.003258,0.386686,-0.093546,-0.177152,0.280647,-0.556676,0.267730,...,-0.329821,0.188621,0.107821,0.146280,0.266547,-0.132216,0.083194,0.331464,0.399364,0
4,0.041432,0.458728,0.125541,-0.458067,-0.359498,-0.516204,0.612352,0.468278,0.044442,-0.177572,...,-0.072933,-0.018716,-0.308049,0.574486,0.226451,-0.231072,-0.202866,0.580723,0.638694,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-0.259413,-0.284309,-0.201027,-0.017551,-0.449832,-0.614521,-0.027409,0.587838,-0.205535,0.024848,...,-0.291052,0.299488,-0.591564,0.097257,1.108976,-0.219779,-0.332158,0.165074,0.691611,0
96,0.333628,0.137654,-0.122666,0.026730,-0.218588,-0.236194,-0.025301,0.512384,-0.221927,0.041977,...,0.123007,0.256707,-0.086893,0.220921,0.204233,-0.049905,-0.264247,0.445555,0.525016,0
97,-0.048696,0.212514,-0.015065,-0.160278,-0.397783,-0.397764,0.431256,0.466553,0.138425,-0.281985,...,-0.322410,0.049418,-0.418673,-0.008705,-0.012006,-0.205094,-0.179316,0.340955,0.728311,0
98,-0.494844,-0.336033,0.245763,-0.179141,-0.247788,-0.195357,0.084165,0.652698,-0.487815,-0.468103,...,-0.099521,-0.098415,-0.173609,0.118502,0.019956,-0.025561,-0.542009,0.321918,0.656737,0


Код выше, показывает механизм преобразования эмбеддингов на маленькой выборке. То же самое было расчитано на всем датасете. Прочитаем его.

#### Выгрузка датасета

In [53]:
# df_ed.to_csv('df_ed_20k.csv', index=False)

Выгружаем датасет после этапа преобразования в эмбеддинги, для того, чтобы не пересчитывать их в следующий сеанс или на другом устройстве. Выгрузка датасета закомментирована, она не должна больше исполняться.

#### Загрузка датасета

Рассчитанные эмбендинги в Colab я сохранил в Яндекс.Диске. Здесь сохранен неполный датасет, порядка 78% от исходного. Прочитаем `csv-файл`.

In [54]:
# используем api 
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?' 
public_key = 'https://disk.yandex.ru/d/KLot7E--emZcKg' 
 
# получаем url 
final_url = base_url + urlencode(dict(public_key=public_key)) 
response = requests.get(final_url) 
download_url = response.json()['href'] 
 
# загружаем файл в df 
download_response = requests.get(download_url)
df_ed_78 = pd.read_csv(download_url, sep=',')
df_ed_78

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,toxic
0,-0.214180,0.147004,-0.198792,0.039257,-0.117727,-0.340003,-0.144716,0.312527,-0.208113,-0.603441,...,-0.209616,0.381346,0.129333,0.156410,-0.196734,-0.076245,-0.302536,0.222647,0.358534,0
1,0.352702,0.166299,0.279629,-0.027249,0.170146,-0.320978,0.132645,0.255978,-0.076715,-0.255779,...,-0.071787,-0.004774,-0.050075,0.283952,0.130654,-0.132127,-0.168120,0.573513,0.351487,1
2,-0.390808,0.389703,0.201352,-0.158126,-0.351628,-0.587400,0.745436,0.543038,0.301617,-0.075048,...,-0.401640,0.525003,-0.361147,0.151722,0.284467,-0.114487,-0.712322,0.500326,0.233124,1
3,-0.172516,-0.347529,0.091326,0.274101,-0.638771,-0.430128,0.085095,0.347237,-0.185096,-0.064509,...,0.003850,0.156936,-0.172188,-0.043264,0.498429,-0.266780,-0.493660,0.516713,0.692796,0
4,-0.525008,-0.264399,-0.078045,-0.365558,-0.029540,-0.140087,0.207137,0.112012,-0.148200,-0.204890,...,-0.169026,-0.345219,-0.417685,-0.296221,-0.003516,-0.313758,-0.492042,0.370257,0.604431,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121363,-0.212598,0.256898,-0.447700,0.227991,0.286868,-0.134953,0.286301,0.290358,-0.119520,-0.162863,...,0.228032,0.299517,-0.117757,0.055547,0.581419,-0.358383,-0.404726,0.595774,0.296562,0
121364,0.159202,0.578980,0.106536,-0.247540,-0.566379,-0.770876,0.918120,0.894100,0.635024,-0.554825,...,-0.121072,0.448461,-0.421211,-0.195475,0.213188,-0.265779,-0.492184,0.749388,0.467811,1
121365,-0.183408,-0.071542,-0.303102,-0.209128,-0.538787,-0.419055,0.380409,0.182944,0.382511,-0.468929,...,-0.184913,-0.053646,-0.508643,-0.307283,0.272276,-0.089274,0.003302,0.500062,0.727596,0
121366,-0.865203,-0.198704,-0.309393,-0.249649,-0.151374,0.112604,0.326239,0.257939,-0.153978,-0.107524,...,0.156371,-0.256471,-0.130817,0.261458,-0.010640,-0.337773,-0.589161,0.084944,0.195185,0


In [55]:
df_ed_78 = df_ed_78.astype('float32') #меняем тип - чтобы при работе с данными требовалось меньше памяти
df_ed_78['toxic'] = df_ed_78['toxic'].astype('int')

### Обучение для BERT

#### Ограничение датасета

Так как датасет получается тяжелым, чтобы была возможность на его данных провести обучение, их нужно немного разделить. Сделаем это с помощью `train_test_split`, чтобы сохранить соотношение в целевом признаке.

Таблица `out` - это таблица неиспользуемых данных. Их слишком много, чтобы оборудование было способно их обработать.

Соотношение 0.1 - это 10% на второй сет в сплите, то есть порядка 12 тысяч строк будет при таком соотношении во втором датасете.

In [56]:
out, df_ed = train_test_split(df_ed_78, test_size=0.1, random_state=RANDOM_STATE, stratify= df_ed_78['toxic']) 

In [57]:
df_ed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12137 entries, 39465 to 20976
Columns: 769 entries, 0 to toxic
dtypes: float32(768), int32(1)
memory usage: 35.7 MB


#### Разбиение данных

In [58]:
features_train, features_test, target_train, target_test = train_test_split( #создаем 4 датасета, два признаков (тест+валидация) и два целевых, 
    df_ed.drop(columns='toxic'), #для датасетов признаков удаляем целевой
    df_ed['toxic'], #для целевого оставляем только целевой
    test_size=0.2, #с соотношением 
    random_state=RANDOM_STATE, #с заданной опорой для рандома 
    stratify= df_ed['toxic']) #с заданной стратификацией по целевому признаку

#### Модель логистической регрессии

In [59]:
%%time

log_reg = LogisticRegression(solver='lbfgs', max_iter=1000, class_weight= 'balanced') #максимальное количество итераций 1000, 
log_reg_scores = cross_val_score(log_reg, features_train, target_train.values, scoring='f1', cv=10)
log_reg_f1 = log_reg_scores.mean()

print("Среднее значение метрики F1 для модели логистической регрессии "\
    "с использованием кросс-валидации:", log_reg_f1)

Среднее значение метрики F1 для модели логистической регрессии с использованием кросс-валидации: 0.6284264732021358
CPU times: total: 56.1 s
Wall time: 26.5 s


#### Модель опорных векторов

In [60]:
%%time

svclassifier = SVC(kernel='linear') 
svclassifier_scores = cross_val_score(svclassifier, features_train, target_train.values, scoring='f1', cv=10)
svclassifier_f1 = svclassifier_scores.mean()

print("Среднее значение метрики F1 для модели SVC "\
    "с использованием кросс-валидации:", svclassifier_f1)

Среднее значение метрики F1 для модели SVC с использованием кросс-валидации: 0.6631569448391847
CPU times: total: 45.5 s
Wall time: 49 s


#### Модель случайного леса

In [61]:
%%time

forest = RandomForestClassifier(random_state=RANDOM_STATE, class_weight= 'balanced') #модель случайного леса
parameters = {'n_estimators': range (1, 300), 'max_depth': range (1, 50)} #перебор гиперпараметров
#применение метода гридсёрч со встроенной кросс-валидацией к модели леса с перебором указанных параметров

randomized_forest = RandomizedSearchCV(forest, n_iter=20, param_distributions= parameters, scoring='f1', n_jobs= -3, cv=5)
#обучение модели
randomized_forest.fit(features_train, target_train.values)

#лучшее значение после перебора параметров 
best_forest = abs(randomized_forest.best_score_)

print("Лучшие параметры для модели случайного леса с "\
    "использованием кросс-валидации:", randomized_forest.best_params_)
print("Наибольшее значение метрики F1 для модели случайного леса "\
    "при лучших гиперпараметрах с использованием кросс-валидации:", best_forest)

Лучшие параметры для модели случайного леса с использованием кросс-валидации: {'n_estimators': 106, 'max_depth': 9}
Наибольшее значение метрики F1 для модели случайного леса при лучших гиперпараметрах с использованием кросс-валидации: 0.6142330101451825
CPU times: total: 11 s
Wall time: 2min 34s


#### LightGBM

In [84]:
%%time

lgbm = lgb.LGBMClassifier() #
parameters = {'n_estimators': range (1, 1000), 'max_depth': range (1, 100), 'learning_rate': np.arange(0.05, 0.5, 0.05)}
#применение метода гридсёрч со встроенной кросс-валидацией

rand_lgbm = RandomizedSearchCV(lgbm, n_iter=20, param_distributions= parameters, scoring='f1', n_jobs= -3, cv=5)
#обучение модели
rand_lgbm.fit(features_train, target_train.values)

#лучшее значение после перебора параметров 
best_lgbm = rand_lgbm.best_score_

print("Лучшие параметры для модели LGBMClassifier с "\
    "использованием кросс-валидации:", rand_lgbm.best_params_)
print("Наибольшее значение метрики F1 для модели LGBMClassifier "\
    "при лучших гиперпараметрах с использованием кросс-валидации:", best_lgbm)

Лучшие параметры для модели случайного леса с использованием кросс-валидации: {'n_estimators': 617, 'max_depth': 97, 'learning_rate': 0.3}
Наибольшее значение метрики F1 для модели LGBMClassifier при лучших гиперпараметрах с использованием кросс-валидации: 0.6463309945210048
CPU times: total: 1min 20s
Wall time: 7min 19s


### Выводы по методу BERT

In [87]:
models = ['Логистическая регрессия', 'Модель опорных векторов', 'Случайный лес', 'LGBMClassifier']
F1 = [log_reg_f1, svclassifier_f1, best_forest, best_lgbm]
total_time = ['~30 секунд', '~1 минута', '~3 минуты', '~10 минут']

bert_result_table = pd.DataFrame({ #созаем датафрейм
    'Модель': models, #
    'Значение метрики F1': F1,
    'Время расчета (~12 000 строк)': total_time
}) #
bert_result_table

,Модель,Значение метрики F1,Время расчета (~12 000 строк)
0,Логистическая регрессия,0.628426,~30 секунд
1,Модель опорных векторов,0.663157,~1 минута
2,Случайный лес,0.614233,~3 минуты
3,LGBMClassifier,0.646331,~10 минут


Мы получили довольно неплохие показатели метрики F1. На всех моделях удалось получить сравнительно одинаковое значение метрики. На модели опорных векторов получили максимальное значение 0.66. Чего, в свою очередь недостаточно, для выполнения этой задачи.

Модель `LGBMClassifier` дает результат значительнее дольше остальных моделей.

## TF-IDF

Оценка важности слова определяется величиной `TF-IDF`. `TF` отвечает за количество упоминаний слова в отдельном тексте, а `IDF` отражает частоту его употребления во всём корпусе.

### Подготовка

#### Подгрузка необходимых данных

In [63]:
dft = df.copy() #исходный датасет, до преобразований BERTa
df_check = df.head(100) #таблица для проверки работоспособности лемматизатора

In [64]:
#загружаем необходимые дополнения для библиотеки
nltk.download('wordnet') 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords') 

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\maxpe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maxpe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\maxpe\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maxpe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [65]:
pd.options.mode.chained_assignment = None #убираем предупреждения

#### Функция очистки текста

In [66]:
def clear_text(text):

    def limit(text):
        #убираем из комментариев слова длиной от 50 до 10000 символов
        #это ограничение необходимо, т.к. есть "слово", ломающее лемматизатор
        lim = re.sub(r'\b\w{50,10000}\b', ' ', text) 
        lim1 = " ".join(lim.split()) #применяем фильтр
        
        return lim1

    clear = re.sub(r'[^a-zA-Z ]', ' ', limit(text)) #фильтруем латиницу, символы
    clear1 = " ".join(clear.split()) #применяем фильтр
    
    return clear1

In [67]:
dft['lemma'] = dft['text'].apply(clear_text) #применяем очистку текста

#### Лемматизатор

In [68]:
def lemmatize_text(text):
    #лемматизируем текст с помощью лемматизатора предложений из пакета pywsd
    lemma = lemmatize_sentence(text) 
    return " ".join(lemma)

Лемматизатор, так же как и преобразование эмбеддингов, считается довольно долго, порядка 50 минут. Ниже лемматизирован кусочек из 100 строк оригинальной таблицы, чтобы показать работу лемматизатора.

Полный датасет рассчитан заранее и загружается после проверки.

In [69]:
df_check['lemma'] = df_check['text'].apply(clear_text) #повторяем очистку, т.к. датасет - другой
df_check['lemma'] = df_check['lemma'].apply(lemmatize_text) #лемматизируем текст
df_check.head()

,text,toxic,lemma
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not try to edit war it s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


#### Выгрузка датасета

Так же, как и с таблицей после преобразования эмбеддингов `df_ed`, этот датасет был выгружен один раз, после чего вывод таблицы - закомментирован. 

Лемматизировался полный датасет `df`. 

In [6]:
# dft.to_csv('df_lemma.csv', index=False)

#### Загрузка лемматизированного датасета

In [96]:
# используем api 
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?' 
public_key = 'https://disk.yandex.ru/d/zNN-U7f8m-UaaQ' 
 
# получаем url 
final_url = base_url + urlencode(dict(public_key=public_key)) 
response = requests.get(final_url) 
download_url = response.json()['href'] 
 
# загружаем файл в df 
download_response = requests.get(download_url)
df_lem_f = pd.read_csv(download_url, sep=',')
df_lem_f

,text,toxic,lemma
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not try to edit war it s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...
...,...,...,...
159287,""":::::And for the second time of asking, when ...",0,and for the second time of ask when your view ...
159288,You should be ashamed of yourself \n\nThat is ...,0,you should be ashamed of yourself that be a ho...
159289,"Spitzer \n\nUmm, theres no actual article for ...",0,spitzer umm theres no actual article for prost...
159290,And it looks like it was actually you who put ...,0,and it look like it be actually you who put on...


### Обучение для TF-IDF

#### Разбиение данных

Заранее посчитав все метрики на полном датасете, я пришел к выводу, что датасет придется также ограничить, взяв на разбиение признаков 60% от исходного. Время обучения некоторых моделей даже при грубом переборе гиперпараметров (а в случае модели опорных векторов, то даже на голой модели без перебора) слишком велико, притом, для получения необходимого порога метрики хватает с запасом 60% данных. 

Обязательно стратифицируем по целевому признаку, чтобы сохранить баланс классов целевого признака.

In [108]:
out1, df_lem = train_test_split(df_lem_f, test_size=0.6, random_state=RANDOM_STATE, stratify= df_lem_f['toxic']) 

In [109]:
features_train_lem, features_test_lem, target_train_lem, target_test_lem = train_test_split( #создаем 4 датасета, два признаков (тест+валидация) и два целевых, 
    df_lem.drop(columns='toxic'), #для датасетов признаков удаляем целевой
    df_lem['toxic'], #для целевого оставляем только целевой
    test_size=0.2, #с соотношением 
    random_state=RANDOM_STATE, #с заданной опорой для рандома 
    stratify= df_lem['toxic']) #с заданной стратификацией по целевому признаку

#### Вводим стоп слова

In [110]:
stop_words = set(stopwords.words('english')) 

Стоп слова - слова, не несущие смысловой нагрузки. Они не должны влиять на обучение.

#### Мешки слов

Создадим счётчик, далее передаём счётчику корпус текстов. Для этого вызовем `fit_transform()`. 

Счётчик выделит из корпуса уникальные слова и посчитает количество их вхождений в каждом тексте корпуса. Отдельные буквы счётчик как слова не учитывает.

In [111]:
corpus_train = features_train_lem['lemma'].values.astype('U') #создаем корпус текстов
count_tf_idf_train = TfidfVectorizer(stop_words=stop_words) #создаем счетчик величин TF-IDF, с учетом стоп слов
tf_idf_train = count_tf_idf_train.fit_transform(corpus_train) #передаем счётчику корпус текстов (fit запускается только на обучающей выборке)
tf_idf_train.shape

(76460, 99373)

In [112]:
# count_tf_idf_train.get_feature_names() #посмотреть словарь уникальных слов

Аналогичным образом работаем и с тестовой выборкой. Единственное отличие - `transform` для теста не обучается.

In [113]:
corpus_test = features_test_lem['lemma'].values.astype('U')
tf_idf_test = count_tf_idf_train.transform(corpus_test)
tf_idf_test.shape

(19116, 99373)

Данные предобработаны, можно переходить к обучению.

### Обучение

#### Модель логистической регрессии

In [114]:
%%time

log_reg_lem = LogisticRegression(solver='lbfgs', max_iter=1000, class_weight= 'balanced') #максимальное количество итераций 1000, 
log_reg_scores_lem = cross_val_score(log_reg_lem, tf_idf_train, target_train_lem.values, scoring='f1', cv=10)
log_reg_f1_lem = log_reg_scores_lem.mean()

print("Среднее значение метрики F1 для модели логистической регрессии "\
    "с использованием кросс-валидации:", log_reg_f1_lem)

Среднее значение метрики F1 для модели логистической регрессии с использованием кросс-валидации: 0.7400516020260364
CPU times: total: 4.52 s
Wall time: 15.6 s


#### Модель опорных векторов

In [115]:
%%time

svclassifier_lem = SVC(kernel='linear', class_weight= 'balanced') 
svclassifier_scores_lem = cross_val_score(svclassifier_lem, tf_idf_train, target_train_lem.values, scoring='f1', cv=2)
svclassifier_f1_lem = svclassifier_scores_lem.mean()

print("Среднее значение метрики F1 для модели SVC "\
    "с использованием кросс-валидации:", svclassifier_f1_lem)

Среднее значение метрики F1 для модели SVC с использованием кросс-валидации: 0.7394661035823685
CPU times: total: 8min 3s
Wall time: 8min 33s


#### Модель случайного леса

In [116]:
%%time

forest_lem = RandomForestClassifier(random_state=RANDOM_STATE, class_weight= 'balanced') #модель случайного леса
parameters = {'n_estimators': range (1, 500, 50), 'max_depth': range (1, 100, 10)} #перебор гиперпараметров
#применение метода гридсёрч со встроенной кросс-валидацией к модели леса с перебором указанных параметров

randomized_forest_lem = RandomizedSearchCV(forest_lem, n_iter=15, param_distributions= parameters, scoring='f1', n_jobs= -2, cv=2)
#обучение модели
randomized_forest_lem.fit(tf_idf_train, target_train_lem.values)

#лучшее значение после перебора параметров 
best_forest_lem = abs(randomized_forest_lem.best_score_)

print("Лучшие параметры для модели случайного леса с "\
    "использованием кросс-валидации:", randomized_forest_lem.best_params_)
print("Наибольшее значение метрики F1 для модели случайного леса "\
    "при лучших гиперпараметрах с использованием кросс-валидации:", best_forest_lem)

Лучшие параметры для модели случайного леса с использованием кросс-валидации: {'n_estimators': 151, 'max_depth': 91}
Наибольшее значение метрики F1 для модели случайного леса при лучших гиперпараметрах с использованием кросс-валидации: 0.5342654316870761
CPU times: total: 1min 12s
Wall time: 5min 52s


#### LightGBM

In [117]:
%%time

lgbm_lem = lgb.LGBMClassifier() #
parameters = {'n_estimators': range (1, 500, 10), 'max_depth': range (1, 100, 5), 'learning_rate': np.arange(0.1, 0.8, 0.1)}
#применение метода гридсёрч со встроенной кросс-валидацией

rand_lgbm_lem = RandomizedSearchCV(lgbm_lem, n_iter=20, param_distributions= parameters, scoring='f1', n_jobs= -2, cv=3)
#обучение модели
rand_lgbm_lem.fit(tf_idf_train, target_train_lem.values)

#лучшее значение после перебора параметров 
best_lgbm_lem = rand_lgbm_lem.best_score_

print("Лучшие параметры для модели LGBMClassifier с "\
    "использованием кросс-валидации:", rand_lgbm_lem.best_params_)
print("Наибольшее значение метрики F1 для модели LGBMClassifier "\
    "при лучших гиперпараметрах с использованием кросс-валидации:", best_lgbm_lem)

Лучшие параметры для модели случайного леса с использованием кросс-валидации: {'n_estimators': 131, 'max_depth': 96, 'learning_rate': 0.30000000000000004}
Наибольшее значение метрики F1 для модели LGBMClassifier при лучших гиперпараметрах с использованием кросс-валидации: 0.758343798605508
CPU times: total: 1min 30s
Wall time: 5min 45s


### Выводы по методу TF-IDF

In [120]:
models = ['Логистическая регрессия', 'Модель опорных векторов', 'Случайный лес', 'LGBMClassifier']
F1 = [log_reg_f1_lem, svclassifier_f1_lem, best_forest_lem, best_lgbm_lem]
total_time = ['~15 секунд', '~8 минут', '~6 минут', '~6 минут']

tf_result_table = pd.DataFrame({ #созаем датафрейм
    'Модель': models, #
    'Значение метрики F1': F1,
    'Время расчета (~12 000 строк)': total_time
}) #
tf_result_table

,Модель,Значение метрики F1,Время расчета (~12 000 строк)
0,Логистическая регрессия,0.740052,~15 секунд
1,Модель опорных векторов,0.739466,~8 минут
2,Случайный лес,0.534265,~6 минут
3,LGBMClassifier,0.758344,~6 минут


Мы получили два близких значения метрики F1 к требуемому: на логистической регрессии и на модели опорных векторов. Несмотря на то, что значение метрики для регрессии ниже требуемого, она все же может пригодиться, если нужно сделать довольно точные предсказания на скорость. Модель векторов, напротив, самая медленная из всех. Модель леса оказалась самой неточной. 

Единственная модель, удовлетворяющая условию - `LGBMClassifier`, дает необходимое значение метрики, к тому же предсказывает результат не дольше других моделей (а на полном наборе - быстрее других).

### Проверяем лучшую модель

Тестируем модель `LGBMClassifier`, которая при предобработке методом `TF-IDF`, показала себя как точную и достаточно быструю (даже с учетом перебора гиперпараметров) модель.

In [121]:
%%time

lgbm_lem_1 = lgb.LGBMClassifier(
    n_estimators=rand_lgbm_lem.best_params_['n_estimators'], 
    max_depth=rand_lgbm_lem.best_params_['max_depth'], 
    learning_rate=rand_lgbm_lem.best_params_['learning_rate'])
lgbm_lem_1.fit(tf_idf_train, target_train_lem.values)

predictions_lgbm_lem_1 = lgbm_lem_1.predict(tf_idf_test)

lgbm_lem_f1 = f1_score(target_test_lem, predictions_lgbm_lem_1)
print("Наименьшее значение метрики F1 на тестовых данных", lgbm_lem_f1)

Наименьшее значение метрики F1 на тестовых данных 0.7659820653746022
CPU times: total: 1min 32s
Wall time: 6.95 s


Получили значение метрики F1 в пределах 0,77.

## Выводы

В проекте было рассмотрено два метода обработки натурального языка, а именно:

- `BERT`,
- `TF-IDF`.

Преобразовав текст для `BERT`, мы получили значение метрики, не удовлетворяющее условию, а именно, метрику F1 < 0.75. Это связано с тем, что модели обучались не на всем наборе данных, а лишь на 10%. Здесь упираемся в аппаратные ограничения и скорость обучения и перебора параметров. 

В методе `TF-IDF` мы добились необходимого значения метрики `F1`, для модели `LGBMClassifier`. Она же показала результат на тестовых данных равный 0.77. Здесь омодели обучались уже на 60% данных, что значительно увеличило значение метрики `F1`. То есть, даже если этот метод менее точен, чем `BERT` (при условии использования 100% данных), он намного менее требователен к аппаратным ресурсам и намного быстрее.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны